<a href="https://colab.research.google.com/github/Hubert26/machine-learning/blob/main/project_hubert_szewczyk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OPIS PROJEKTU

Lending Club to firma pożyczkowa typu peer-to-peer, która łączy pożyczkobiorców z
inwestorami za pośrednictwem platformy internetowej. Obsługuje osoby, które potrzebują
pożyczek osobistych w wysokości od 1000 do 40 000 USD. Pożyczkobiorcy otrzymują pełną
kwotę udzielonej pożyczki pomniejszoną o opłatę początkową, która jest uiszczana firmie.
Inwestorzy kupują weksle zabezpieczone osobistymi pożyczkami i płacą Lending Club
opłatę za usługę. Firma Lending Club udostępnia dane o wszystkich pożyczkach
udzielonych za pośrednictwem swojej platformy w określonych okresach.
Na potrzeby tego projektu zostały użyte dane dotyczące pożyczek udzielonych za
pośrednictwem Lending Club na przestrzeni lat 2007 -2011. Każda pożyczka jest opatrzona
informacją o tym, czy ostatecznie została spłacona (Fully Paid lub Charged off w kolumnie
loan_status).

W tym projekcie chcę zbudować model klasyfikacyjny, który na podstawie zebranych danych będzie przewidywał, czy potencjalny pozyczkobiorca spłaci swój dług.

#KOD

##Import bibliotek i ustawienia

In [144]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

#Ustawienie braku maksymalnej ilości wyświetlanych kolumn
pd.options.display.max_columns = None

##Definicje funkcji

In [145]:
def display_info(df, selected_columns, df_info):
    for col_name in selected_columns:
        print("Column Name:", col_name)

        # Description
        if col_name in df_info.index:
            print("Description:", df_info.loc[col_name]['Description'])
        else:
            print("Description not available.")

        # Statistics
        print("\nStatistics:")
        print(df[col_name].describe())

        # Value counts
        print("\nValue counts:")
        print(df[col_name].value_counts())

        # Unique values
        print("\nUnique values:")
        print(df[col_name].unique())

        # Mode
        print("\nMode:")
        print(df[col_name].mode())

        # NaN count
        nan_count = df[col_name].isna().sum()
        print("\nNaN count:", nan_count)

        print("\n" + "=" * 40)  # Separator between columns

##1. Obróbka danych

###Pobranie danych z plików

In [146]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/machine-learning/main/Loan_data.csv', low_memory=False)

In [147]:
df = df_raw.copy()

In [148]:
df_info_raw = pd.read_csv('https://raw.githubusercontent.com/Hubert26/machine-learning/main/LCDataDictionary.csv')

In [149]:
df_info = df_info_raw.iloc[:, 0:2]
df_info.set_index(df_info.columns.tolist()[0], inplace=True)

###Eksploracja danych

In [150]:
df.shape

(42536, 151)

In [151]:
df.head(2)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1077501,NaN,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,NaN,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.0,0.0,0.00,Jan-2015,171.62,NaN,Aug-2018,739.0,735.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,NaN,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.0,122.9,1.11,Apr-2013,119.66,NaN,Oct-2016,499.0,0.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42536 entries, 0 to 42535
Columns: 151 entries, id to settlement_term
dtypes: float64(120), object(31)
memory usage: 49.0+ MB


In [153]:
df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
count,0.0,42535.000000,42535.000000,42535.000000,42535.000000,4.253100e+04,42535.000000,42506.000000,42535.000000,42535.000000,42506.000000,15609.000000,3651.000000,42506.000000,42506.000000,4.253500e+04,42506.000000,42535.0,42535.0,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42535.000000,42390.0,0.0,42535.0,0.0,0.0,0.0,42506.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42390.0,42506.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41170.000000,42430.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.000000,160.000000,160.000000
mean,NaN,11089.722581,10821.585753,10139.938785,322.623063,6.913656e+04,13.373043,0.152449,713.052545,717.052545,1.081424,35.017618,59.175568,9.343951,0.058156,1.429786e+04,22.124406,0.0,0.0,12020.189644,11312.729377,9675.675936,2239.981444,1.516890,103.015467,14.387843,2613.245652,689.922511,676.952039,0.0,NaN,1.0,NaN,NaN,NaN,0.000094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.143039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.045227,0.000024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4272.137875,49.905875,1.200000
std,NaN,7410.938391,7146.914675,7131.598014,208.927216,6.409635e+04,6.726315,0.512406,36.188439,36.188439,1.527455,22.418427,47.145396,4.496274,0.245713,2.201844e+04,11.592811,0.0,0.0,9094.685888,9038.506549,7105.750304,2585.057393,7.830064,735.799086,162.177837,4385.066535,80.818099,119.647752,0.0,NaN,0.0,NaN,NaN,NaN,0.009700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,29.359579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [154]:
#Sprawdzenie czy wszystkie wartości są unikalne
len(df['id'].unique())==df.shape[0]

True

In [155]:
column_names = df.columns.tolist()

In [156]:
mask = df.isna()

In [157]:
naan_counts_columns = [(mask[column] == True).sum() for column in df.columns]
naan_column = pd.DataFrame([naan_counts_columns], columns=df.columns, index=['NaN'])
df_info = df_info.merge(naan_column.T, left_index=True, right_index=True, how='left')

In [158]:
mode_column = pd.DataFrame(columns=['mode'], index=column_names)
for col_name in column_names:
    if not df[col_name].empty:
        mode_value = df[col_name].mode()
        if not mode_value.empty:
            mode_column.loc[col_name, 'mode'] = mode_value.iloc[0]
        else:
            mode_column.loc[col_name, 'mode'] = None
    else:
        mode_column.loc[col_name, 'mode'] = None
df_info = df_info.merge(mode_column, left_index=True, right_index=True, how='left')

In [159]:
unique_len_column = pd.DataFrame(columns=['unique_len'], index=column_names)
for col_name in column_names:
    if not df[col_name].empty:
        unique_len_column.loc[col_name, 'unique_len'] = len([value for value in df[col_name].unique() if not pd.isna(value)])
    else:
        unique_len_column.loc[col_name, 'unique_len'] = 0
df_info = df_info.merge(unique_len_column, left_index=True, right_index=True, how='left')

In [160]:
df_type_column = pd.DataFrame(columns=['raw_column_type'], index=column_names)
for col_name in column_names:
  df_type_column.loc[col_name, 'raw_column_type'] = df[col_name].dtype

df_info = df_info.merge(df_type_column, left_index=True, right_index=True, how='left')

In [161]:
df_info = df_info.merge(pd.DataFrame(df.describe()).T, left_index=True, right_index=True, how='left')

In [162]:
df_info

,Description,NaN,mode,unique_len,raw_column_type,count,mean,std,min,25%,50%,75%,max
acc_now_delinq,The number of accounts on which the borrower i...,30.0,0.0,2,float64,42506.0,0.000094,0.009700,0.0,0.0,0.0,0.0,1.0
acc_open_past_24mths,Number of trades opened in past 24 months.,42536.0,None,0,float64,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
addr_state,The state provided by the borrower in the loan...,1.0,CA,50,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
all_util,Balance to credit limit on all trades,42536.0,None,0,float64,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
annual_inc,The self-reported annual income provided by th...,5.0,60000.0,5597,float64,42531.0,69136.556420,64096.349719,1896.0,40000.0,59000.0,82500.0,6000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
verification_status,"Indicates if income was verified by LC, not ve...",1.0,Not Verified,3,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
verified_status_joint,Indicates if the co-borrowers' joint income wa...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zip_code,The first 3 numbers of the zip code provided b...,1.0,100xx,837,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###Wyświetlenie Loan Status

Charged Off to status długu uznawanego za nieściągalny po 180 dniach braku płatności, ale nie oznacza umorzenia długu. Odpisanie długu wpływa negatywnie na historię kredytową i zdolność kredytową pożyczkobiorcy. Mimo spłaty długu, status Charged Off pozostaje w raporcie kredytowym przez siedem lat, choć może zostać zmieniony na "odpis opłacony" lub "odpis rozliczony". W praktyce, wiele osób spłaciło dług, ale nadal ma status Charged Off.

In [163]:
df_info.loc["settlement_status"]['Description']

'The status of the borrower’s settlement plan. Possible values are: COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAFT'

In [164]:
df.settlement_status.unique()

array([nan, 'COMPLETE', 'BROKEN', 'ACTIVE'], dtype=object)

In [165]:
df_info.loc["loan_status"]['Description']

'Current status of the loan'

In [166]:
df.loan_status.unique()

array(['Fully Paid', 'Charged Off', nan,
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [167]:
df["loan_status"] = df["loan_status"].replace(
    {"Does not meet the credit policy. Status:Fully Paid": "Fully Paid",
     "Does not meet the credit policy. Status:Charged Off": "Charged Off"})

In [168]:
df[(df.loan_status == 'Charged Off') & (df.settlement_status == "COMPLETE")].shape[0]

142

Wynika z tego, że są 142 wyniki z loan_status == Charged Off wraz z settlement_status == COMPLETE.

W dalszej części zmieniamy wynik Charged Off na Fully Paid dla takich wpisów.

In [169]:
df.loc[df['settlement_status'] == "COMPLETE", 'loan_status'] = "Fully Paid"

In [170]:
df['loan_status'].value_counts()

Fully Paid     36246
Charged Off     6289
Name: loan_status, dtype: int64

###Usunięcie kolumn które zawierają informacje z przyszłości, niedostępne w momencie udzielania pożyczki

In [171]:
future_data = df[["mths_since_last_delinq", "last_pymnt_d",
                      "collection_recovery_fee", "recoveries", "issue_d",
                      "total_rec_late_fee", "total_rec_prncp", "funded_amnt",
                      "funded_amnt_inv", "last_fico_range_high",
                      "last_fico_range_low", "debt_settlement_flag"]]
selected_columns = future_data.columns
df.drop(columns=selected_columns, inplace=True, axis=1, errors='ignore')

###Usunięcie kolumn ze zbędnymi informacjami

In [172]:
selected_columns = df_info[(df_info['NaN'] / df_raw.shape[0]) > 0.9].index.tolist()
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

###Usunięcie kolumn z tylko jedną unikalną wartością

In [173]:
selected_columns = df_info[df_info['unique_len'] < 2].index.tolist()
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

###Usunięcie kolumn typu 'object' o bardzo dużej liczbie wartości z niewielką licznością pojedynczej wartości

In [174]:
df_type_column['raw_column_type'].value_counts()

float64    120
object      31
Name: raw_column_type, dtype: int64

In [175]:
for col_name in df.columns.tolist():
  if df[col_name].dtype == 'object':
    value_counts = df[col_name].value_counts()
    if len(value_counts) > 0.2*df.shape[0] and value_counts.iloc[1] < 10:
      display(value_counts)
      print("\n" + "=" * 40)

1077501    1
538025     1
540325     1
540300     1
540270     1
          ..
770036     1
769217     1
770000     1
765994     1
70686      1
Name: id, Length: 42536, dtype: int64

https://lendingclub.com/browse/loanDetail.action?loan_id=1077501    1
https://lendingclub.com/browse/loanDetail.action?loan_id=538025     1
https://lendingclub.com/browse/loanDetail.action?loan_id=540325     1
https://lendingclub.com/browse/loanDetail.action?loan_id=540300     1
https://lendingclub.com/browse/loanDetail.action?loan_id=540270     1
                                                                   ..
https://lendingclub.com/browse/loanDetail.action?loan_id=770036     1
https://lendingclub.com/browse/loanDetail.action?loan_id=769217     1
https://lendingclub.com/browse/loanDetail.action?loan_id=770000     1
https://lendingclub.com/browse/loanDetail.action?loan_id=765994     1
https://lendingclub.com/browse/loanDetail.action?loan_id=70686      1
Name: url, Length: 42535, dtype: int64

In [176]:
selected_columns = {'url','id'}
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

###Przekształcenie wartości w kolumnach - usunięcie procentów, dodatkowych znaków, itd

In [177]:
df["revol_util"] = df["revol_util"].str[0:-1]
df["int_rate"] = df["int_rate"].str[0:-1]
df["revol_util"].astype(np.float64)
df["int_rate"].astype(np.float64)

df["zip_code"] = df["zip_code"].str[0:-2]

In [178]:
print("df after cleaning:", df.shape)

df after cleaning: (42536, 51)


In [179]:
selected_columns = df.columns.tolist()
for column_name in selected_columns:
    if column_name in df_info.index:
        description = df_info.loc[column_name, 'Description']
        print(f"{column_name} - {description}")
    else:
        print(f"Kolumna '{column_name}' nie istnieje w df_info.")

loan_amnt - The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
term - The number of payments on the loan. Values are in months and can be either 36 or 60.
int_rate - Interest Rate on the loan
installment - The monthly payment owed by the borrower if the loan originates.
grade - LC assigned loan grade
sub_grade - LC assigned loan subgrade
emp_title - The job title supplied by the Borrower when applying for the loan.*
emp_length - Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. 
home_ownership - The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
annual_inc - The self-reported annual income provided by the borrower during registration.
verification_status - Indicates if income was verified b

###Analiza brakujących wartości wraz z ich uzupełnieniem/usunięciem przy wzięciu pod uwagę pewnych ustalonych kryteriów

####"pub_rec"

In [180]:
#Wypełnianie zerami wpisów, w których zmienna pubrec nie zawera informacji o
#negatywnych wpisach w rejestrze publicznym
df["pub_rec"].fillna(0, inplace=True)

####"pub_rec_bankruptcies"

In [181]:
#Wypełnienie zerami kolumny nt. bankructw wpisanych do rejestru publicznego
df["pub_rec_bankruptcies"].fillna(0, inplace=True)

#### "emp_length"

In [182]:
#Uzupełnianie pustych wartości modą
df['emp_length'].fillna(df['emp_length'].mode().iloc[0], inplace=True)

#### "emp_title"

In [183]:
#Z racji dużej ilości pustych wpisów tej cechy, oraz fakt, że jest to zmienna
#typu 'object' oraz ma dużo unikalnych wartości to postanawiam tą zmienną usunąć
display_info(df, ['emp_title'], df_info)
df.drop('emp_title', inplace=True, axis=1)

Column Name: emp_title
Description: The job title supplied by the Borrower when applying for the loan.*

Statistics:
count       39909
unique      30658
top       US Army
freq          139
Name: emp_title, dtype: object

Value counts:
US Army                             139
Bank of America                     115
IBM                                  72
Kaiser Permanente                    61
AT&T                                 61
                                   ... 
Regional Elite Airlines Services      1
Mass General Medical Group            1
Kontera                               1
Southeast Georgia Health ystem        1
Homemaker                             1
Name: emp_title, Length: 30658, dtype: int64

Unique values:
[nan 'Ryder' 'AIR RESOURCES BOARD' ... 'Tanks Tavern'
 'Halping hands company inc.' 'Homemaker']

Mode:
0    US Army
Name: emp_title, dtype: object

NaN count: 2627



#### "open_acc"

In [184]:
display_info(df, ['open_acc'], df_info)
#Uzupełnianie pustych wartości modą
df["open_acc"].fillna(df["open_acc"].mode().iloc[0], inplace=True)

Column Name: open_acc
Description: The number of open credit lines in the borrower's credit file.

Statistics:
count    42506.000000
mean         9.343951
std          4.496274
min          1.000000
25%          6.000000
50%          9.000000
75%         12.000000
max         47.000000
Name: open_acc, dtype: float64

Value counts:
7.0     4252
8.0     4176
6.0     4172
9.0     3922
10.0    3386
5.0     3368
11.0    2944
4.0     2508
12.0    2398
13.0    2060
3.0     1608
14.0    1597
15.0    1290
16.0    1022
17.0     812
2.0      692
18.0     588
19.0     442
20.0     335
21.0     276
22.0     170
23.0     121
24.0      87
25.0      63
26.0      40
1.0       39
27.0      29
28.0      29
30.0      17
29.0      16
31.0      13
34.0       8
32.0       6
33.0       5
35.0       4
36.0       2
38.0       2
37.0       1
46.0       1
39.0       1
42.0       1
41.0       1
44.0       1
47.0       1
Name: open_acc, dtype: int64

Unique values:
[ 3.  2. 10. 15.  9.  7.  4. 11. 14. 12. 20.  8.  

#### "purpose"

In [185]:
display_info(df, ['purpose'], df_info)
df["purpose"].fillna(df["purpose"].mode().iloc[0], inplace=True)

Column Name: purpose
Description: A category provided by the borrower for the loan request. 

Statistics:
count                  42535
unique                    14
top       debt_consolidation
freq                   19776
Name: purpose, dtype: object

Value counts:
debt_consolidation    19776
credit_card            5477
other                  4425
home_improvement       3199
major_purchase         2311
small_business         1992
car                    1615
wedding                1004
medical                 753
moving                  629
house                   426
educational             422
vacation                400
renewable_energy        106
Name: purpose, dtype: int64

Unique values:
['credit_card' 'car' 'small_business' 'other' 'wedding'
 'debt_consolidation' 'home_improvement' 'major_purchase' 'medical'
 'moving' 'vacation' 'house' 'renewable_energy' 'educational' nan]

Mode:
0    debt_consolidation
Name: purpose, dtype: object

NaN count: 1



#### "revol_util"

In [186]:
display_info(df, ['revol_util'], df_info)
df["revol_util"].fillna(df["revol_util"].mode().iloc[0], inplace=True)

Column Name: revol_util
Description: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.

Statistics:
count     42445
unique     1119
top           0
freq       1070
Name: revol_util, dtype: object

Value counts:
0        1070
40.7       65
0.2        64
63         63
66.6       62
         ... 
21.59       1
39.95       1
0.01        1
17.67       1
105.7       1
Name: revol_util, Length: 1119, dtype: int64

Unique values:
['83.7' '9.4' '98.5' ... '103.5' '105.3' '105.7']

Mode:
0    0
Name: revol_util, dtype: object

NaN count: 91



#### "last_credit_pull_d"

In [187]:
display_info(df, ['last_credit_pull_d'], df_info)
df["last_credit_pull_d"].fillna(df["last_credit_pull_d"].mode().iloc[0], inplace=True)

Column Name: last_credit_pull_d
Description: The most recent month LC pulled credit for this loan

Statistics:
count        42531
unique         133
top       Aug-2018
freq          9248
Name: last_credit_pull_d, dtype: object

Value counts:
Aug-2018    9248
Oct-2016    4071
Jul-2018    1233
May-2018     734
Feb-2017     709
            ... 
Feb-2008       2
Dec-2007       2
Jun-2008       1
Jul-2008       1
Jul-2007       1
Name: last_credit_pull_d, Length: 133, dtype: int64

Unique values:
['Aug-2018' 'Oct-2016' 'Jun-2017' 'Apr-2016' 'Apr-2018' 'Feb-2017'
 'Sep-2016' 'Dec-2014' 'Dec-2016' 'Mar-2018' 'Aug-2013' 'Apr-2017'
 'Sep-2017' 'May-2014' 'Jul-2015' 'Feb-2016' 'Jun-2016' 'Jan-2018'
 'Mar-2014' 'Sep-2012' 'Jan-2015' 'Jul-2018' 'Jan-2017' 'Jun-2018'
 'Dec-2015' 'Jun-2012' 'Nov-2017' 'Mar-2015' 'Sep-2014' 'Jan-2016'
 'Mar-2017' 'Apr-2014' 'Oct-2014' 'Feb-2018' 'Feb-2014' 'Feb-2013'
 'Nov-2015' 'Jul-2014' 'Dec-2012' 'Jul-2017' 'Oct-2017' 'Dec-2017'
 'May-2018' 'Jul-2016' 'Jun-2015' 

#### "annual_inc"

In [188]:
display_info(df, ['annual_inc'], df_info)
df["annual_inc"].fillna(df["annual_inc"].mode().iloc[0], inplace=True)

Column Name: annual_inc
Description: The self-reported annual income provided by the borrower during registration.

Statistics:
count    4.253100e+04
mean     6.913656e+04
std      6.409635e+04
min      1.896000e+03
25%      4.000000e+04
50%      5.900000e+04
75%      8.250000e+04
max      6.000000e+06
Name: annual_inc, dtype: float64

Value counts:
60000.0     1591
50000.0     1119
40000.0      935
45000.0      898
30000.0      884
            ... 
108820.0       1
85634.0        1
126996.0       1
57110.0        1
6500.0         1
Name: annual_inc, Length: 5597, dtype: int64

Unique values:
[24000. 30000. 12252. ...  7280. 66624.  6500.]

Mode:
0    60000.0
Name: annual_inc, dtype: float64

NaN count: 5



#### "loan_amnt"

In [189]:
display_info(df, ['loan_amnt'], df_info)
df["loan_amnt"].fillna(df["loan_amnt"].mode().iloc[0], inplace=True)

Column Name: loan_amnt
Description: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.

Statistics:
count    42535.000000
mean     11089.722581
std       7410.938391
min        500.000000
25%       5200.000000
50%       9700.000000
75%      15000.000000
max      35000.000000
Name: loan_amnt, dtype: float64

Value counts:
10000.0    3016
12000.0    2439
5000.0     2260
6000.0     2037
15000.0    2012
           ... 
10350.0       1
19100.0       1
17975.0       1
31150.0       1
20425.0       1
Name: loan_amnt, Length: 898, dtype: int64

Unique values:
[ 5000.  2500.  2400. 10000.  3000.  7000.  5600.  5375.  6500. 12000.
  9000.  1000.  3600.  6000.  9200. 20250. 21000. 15000.  4000.  8500.
  4375. 31825. 12400. 10800. 12500.  9600.  4400. 14000. 11000. 25600.
 16000.  7100. 13000. 17500. 17675.  8000.  3500. 16425.  8200. 20975.
  6400. 14400.  7250. 18000. 35000. 

#### "loan_status"

In [190]:
display_info(df, ['loan_status'], df_info)
df["loan_status"].fillna(df["loan_status"].mode().iloc[0], inplace=True)

Column Name: loan_status
Description: Current status of the loan

Statistics:
count          42535
unique             2
top       Fully Paid
freq           36246
Name: loan_status, dtype: object

Value counts:
Fully Paid     36246
Charged Off     6289
Name: loan_status, dtype: int64

Unique values:
['Fully Paid' 'Charged Off' nan]

Mode:
0    Fully Paid
Name: loan_status, dtype: object

NaN count: 1



#### "fico_range_low"

In [191]:
display_info(df, ['fico_range_low'], df_info)
df["fico_range_low"].fillna(df["fico_range_low"].mode().iloc[0], inplace=True)

Column Name: fico_range_low
Description: The lower boundary range the borrower’s FICO at loan origination belongs to.

Statistics:
count    42535.000000
mean       713.052545
std         36.188439
min        610.000000
25%        685.000000
50%        710.000000
75%        740.000000
max        825.000000
Name: fico_range_low, dtype: float64

Value counts:
685.0    2310
700.0    2267
680.0    2228
695.0    2202
690.0    2196
675.0    1994
705.0    1970
720.0    1949
715.0    1891
725.0    1891
670.0    1854
665.0    1821
710.0    1771
730.0    1670
735.0    1653
660.0    1584
740.0    1490
745.0    1319
750.0    1294
755.0    1100
760.0     984
770.0     799
765.0     792
775.0     654
780.0     573
790.0     422
785.0     406
795.0     338
800.0     256
805.0     193
650.0     131
655.0     127
810.0     125
645.0     112
640.0     102
815.0      28
820.0      19
630.0       6
635.0       5
825.0       3
625.0       2
610.0       2
620.0       1
615.0       1
Name: fico_range_low, dty

#### "fico_range_high"

In [192]:
display_info(df, ['fico_range_high'], df_info)
df["fico_range_high"].fillna(df["fico_range_high"].mode().iloc[0], inplace=True)

Column Name: fico_range_high
Description: The upper boundary range the borrower’s FICO at loan origination belongs to.

Statistics:
count    42535.000000
mean       717.052545
std         36.188439
min        614.000000
25%        689.000000
50%        714.000000
75%        744.000000
max        829.000000
Name: fico_range_high, dtype: float64

Value counts:
689.0    2310
704.0    2267
684.0    2228
699.0    2202
694.0    2196
679.0    1994
709.0    1970
724.0    1949
719.0    1891
729.0    1891
674.0    1854
669.0    1821
714.0    1771
734.0    1670
739.0    1653
664.0    1584
744.0    1490
749.0    1319
754.0    1294
759.0    1100
764.0     984
774.0     799
769.0     792
779.0     654
784.0     573
794.0     422
789.0     406
799.0     338
804.0     256
809.0     193
654.0     131
659.0     127
814.0     125
649.0     112
644.0     102
819.0      28
824.0      19
634.0       6
639.0       5
829.0       3
629.0       2
614.0       2
624.0       1
619.0       1
Name: fico_range_high, 

####fico_range_low oraz fico_range_high

In [193]:
df[['fico_range_low','fico_range_high']].corr()

,fico_range_low,fico_range_high
fico_range_low,1.0,1.0
fico_range_high,1.0,1.0


In [194]:
#Te wartości sa mocno skorelowane więc zamiast dwóch kolumn tworzę jedną z ich średnich
df['fico_mean'] = (df.fico_range_low + df.fico_range_high) / 2
df.drop(['fico_range_low', 'fico_range_high'], axis=1, inplace=True)

###Uzupełnienie modą pozostałych kolumn o NaN < 10

In [195]:
mask = df.isna()
naan_counts_columns = [(mask[column] == True).sum() for column in df.columns]
naan_column = pd.DataFrame([naan_counts_columns], columns=df.columns, index=['NaN'])
display(naan_column)

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_amnt,last_credit_pull_d,verification_status_joint,acc_now_delinq,total_rev_hi_lim,delinq_amnt,pub_rec_bankruptcies,tax_liens,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,fico_mean
NaN,0,1,1,1,1,1,0,1,0,1,0,13294,0,14,1,1,1,30,30,30,0,0,1,0,30,1,1,1,1,0,42536,30,42536,30,0,106,42536,42536,42536,42536,42536,42536,42536,42536,42536,42536,42536,42536,0


In [213]:
selected_columns = [col_name for col_name in naan_column.columns if ((naan_column[col_name] > 0) & (naan_column[col_name] < 10)).all()]
for col_name in selected_columns:
    if df[col_name].dtype == 'object':
        df[col_name].fillna(df[col_name].mode().iloc[0], inplace=True)
    elif df[col_name].dtype == np.float64:
        df[col_name].fillna(df[col_name].mean(), inplace=True)


In [196]:
selected_columns = [col_name for col_name in naan_column.columns if ((naan_column[col_name] > 10) & (naan_column[col_name] < 0.1*df.shape[0])).all()]

####"title"

In [197]:
display_info(df, ["title"], df_info)
df.drop("title", inplace=True, axis=1, errors='ignore')

Column Name: title
Description: The loan title provided by the borrower

Statistics:
count                  42522
unique                 21264
top       Debt Consolidation
freq                    2259
Name: title, dtype: object

Value counts:
Debt Consolidation                  2259
Debt Consolidation Loan             1760
Personal Loan                        708
Consolidation                        547
debt consolidation                   532
                                    ... 
CitiCard PayOff                        1
Taxes Loan                             1
Blazing in 5 years                     1
I was scammed and now recovering       1
Aroundthehouse                         1
Name: title, Length: 21264, dtype: int64

Unique values:
['Computer' 'bike' 'real estate business' ... 'delight' 'Car repair bill'
 'Aroundthehouse']

Mode:
0    Debt Consolidation
Name: title, dtype: object

NaN count: 14



####"delinq_2yrs"

####"earliest_cr_line"

In [198]:
display_info(df, ["earliest_cr_line"], df_info)
df["earliest_cr_line"].fillna(df["earliest_cr_line"].mode().iloc[0], inplace=True)

Column Name: earliest_cr_line
Description: The month the borrower's earliest reported credit line was opened

Statistics:
count        42506
unique         530
top       Oct-1999
freq           393
Name: earliest_cr_line, dtype: object

Value counts:
Oct-1999    393
Nov-1998    390
Oct-2000    370
Dec-1998    366
Dec-1997    348
           ... 
Jul-1964      1
Mar-1963      1
Nov-1965      1
Apr-1966      1
Nov-1959      1
Name: earliest_cr_line, Length: 530, dtype: int64

Unique values:
['Jan-1985' 'Apr-1999' 'Nov-2001' 'Feb-1996' 'Jan-1996' 'Nov-2004'
 'Jul-2005' 'Jan-2007' 'Apr-2004' 'Sep-2004' 'Jan-1998' 'Oct-1989'
 'Jul-2003' 'May-1991' 'Sep-2007' 'Oct-1998' 'Aug-1993' 'Oct-2003'
 'Jan-2001' 'Nov-1997' 'Feb-1983' 'Jul-1985' 'Apr-2003' 'Jun-2001'
 'Feb-2002' 'Aug-1984' 'Nov-2006' 'Dec-1987' 'Nov-1981' 'Feb-1997'
 'Apr-2005' 'Oct-2007' 'Dec-2000' 'Apr-2007' 'Dec-2001' 'Jan-2003'
 'Mar-1994' 'Sep-1998' 'Jun-2004' 'Nov-1995' 'Jul-1999' 'Jun-1995'
 'Sep-1992' 'Jan-2002' 'Apr-1992' 'Oct

In [199]:
display_info(df, ["delinq_2yrs"], df_info)
df["delinq_2yrs"].fillna(df["delinq_2yrs"].mode().iloc[0], inplace=True)

Column Name: delinq_2yrs
Description: The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years

Statistics:
count    42506.000000
mean         0.152449
std          0.512406
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         13.000000
Name: delinq_2yrs, dtype: float64

Value counts:
0.0     37771
1.0      3595
2.0       771
3.0       244
4.0        72
5.0        27
6.0        13
7.0         6
8.0         3
11.0        2
9.0         1
13.0        1
Name: delinq_2yrs, dtype: int64

Unique values:
[ 0.  2.  3.  1.  4.  6.  5.  8.  7.  9. 11. nan 13.]

Mode:
0    0.0
Name: delinq_2yrs, dtype: float64

NaN count: 30



####"inq_last_6mths"

In [200]:
display_info(df, ["inq_last_6mths"], df_info)
df["inq_last_6mths"].fillna(df["inq_last_6mths"].mode().iloc[0], inplace=True)

Column Name: inq_last_6mths
Description: The number of inquiries in past 6 months (excluding auto and mortgage inquiries)

Statistics:
count    42506.000000
mean         1.081424
std          1.527455
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max         33.000000
Name: inq_last_6mths, dtype: float64

Value counts:
0.0     19657
1.0     11247
2.0      5987
3.0      3182
4.0      1056
5.0       596
6.0       339
7.0       182
8.0       115
9.0        50
10.0       24
11.0       15
12.0       15
15.0        9
13.0        6
14.0        6
18.0        4
16.0        3
17.0        2
24.0        2
19.0        2
32.0        1
33.0        1
31.0        1
28.0        1
25.0        1
27.0        1
20.0        1
Name: inq_last_6mths, dtype: int64

Unique values:
[ 1.  5.  2.  0.  3.  4.  6.  7.  8. nan  9. 10. 11. 12. 15. 14. 33. 17.
 32. 24. 13. 18. 16. 31. 28. 25. 27. 20. 19.]

Mode:
0    0.0
Name: inq_last_6mths, dtype: float64

NaN count: 30



####"total_acc"

In [201]:
display_info(df, ["total_acc"], df_info)
df["total_acc"].fillna(df["total_acc"].mode().iloc[0], inplace=True)

Column Name: total_acc
Description: The total number of credit lines currently in the borrower's credit file

Statistics:
count    42506.000000
mean        22.124406
std         11.592811
min          1.000000
25%         13.000000
50%         20.000000
75%         29.000000
max         90.000000
Name: total_acc, dtype: float64

Value counts:
15.0    1552
16.0    1547
17.0    1543
14.0    1531
20.0    1504
        ... 
74.0       1
77.0       1
78.0       1
72.0       1
90.0       1
Name: total_acc, Length: 83, dtype: int64

Unique values:
[ 9.  4. 10. 37. 38. 12. 11. 13.  3. 23. 34. 29. 28. 42. 14. 22. 21. 17.
  7. 31. 44. 26. 16.  6. 18. 27. 24. 25. 40. 35.  8. 20. 15. 19. 36. 51.
 32. 30. 33. 46.  5. 61. 56. 50. 41. 39. 79. 62. 43. 47. 53. 45. 60. 55.
 52. 58. 54. 57. 49. 63. 48. 59. 77. 87. 75. 72. 64. 67. 78. 76. 74. 66.
 81. 90. 80. 71. 69. 73. 70. 68. 65.  2. nan  1.]

Mode:
0    15.0
Name: total_acc, dtype: float64

NaN count: 30



####"acc_now_delinq"

In [202]:
display_info(df, ["acc_now_delinq"], df_info)
df.drop("acc_now_delinq", inplace=True, axis=1, errors='ignore')

Column Name: acc_now_delinq
Description: The number of accounts on which the borrower is now delinquent.

Statistics:
count    42506.000000
mean         0.000094
std          0.009700
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: acc_now_delinq, dtype: float64

Value counts:
0.0    42502
1.0        4
Name: acc_now_delinq, dtype: int64

Unique values:
[ 0. nan  1.]

Mode:
0    0.0
Name: acc_now_delinq, dtype: float64

NaN count: 30



####"delinq_amnt"

In [203]:
display_info(df, ["delinq_amnt"], df_info)
df.drop("delinq_amnt", inplace=True, axis=1, errors='ignore')

Column Name: delinq_amnt
Description: The past-due amount owed for the accounts on which the borrower is now delinquent.

Statistics:
count    42506.000000
mean         0.143039
std         29.359579
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max       6053.000000
Name: delinq_amnt, dtype: float64

Value counts:
0.0       42504
27.0          1
6053.0        1
Name: delinq_amnt, dtype: int64

Unique values:
[   0.   nan   27. 6053.]

Mode:
0    0.0
Name: delinq_amnt, dtype: float64

NaN count: 30



####"tax_liens"

In [204]:
display_info(df, ["tax_liens"], df_info)
df.drop("tax_liens", inplace=True, axis=1, errors='ignore')

Column Name: tax_liens
Description: Number of tax liens

Statistics:
count    42430.000000
mean         0.000024
std          0.004855
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: tax_liens, dtype: float64

Value counts:
0.0    42429
1.0        1
Name: tax_liens, dtype: int64

Unique values:
[ 0. nan  1.]

Mode:
0    0.0
Name: tax_liens, dtype: float64

NaN count: 106



#### Usunięcie pozostałych kolumn o dużej ilości NaN

In [211]:
selected_columns = [col_name for col_name in naan_column.columns if (naan_column[col_name] >  0.1*df.shape[0]).all()]
display_info(df, selected_columns, df_info)
df.drop(selected_columns, inplace=True, axis=1, errors='ignore')

Column Name: verification_status_joint
Description not available.

Statistics:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: verification_status_joint, dtype: float64

Value counts:
Series([], Name: verification_status_joint, dtype: int64)

Unique values:
[nan]

Mode:
Series([], Name: verification_status_joint, dtype: float64)

NaN count: 42536

Column Name: total_rev_hi_lim
Description not available.

Statistics:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: total_rev_hi_lim, dtype: float64

Value counts:
Series([], Name: total_rev_hi_lim, dtype: int64)

Unique values:
[nan]

Mode:
Series([], Name: total_rev_hi_lim, dtype: float64)

NaN count: 42536

Column Name: revol_bal_joint
Description not available.

Statistics:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: revol_bal_joint, dtype: float64



###Podsumowanie


In [214]:
mask = df.isna()
naan_counts_columns = [(mask[column] == True).sum() for column in df.columns]
naan_column = pd.DataFrame([naan_counts_columns], columns=df.columns, index=['NaN'])
display(naan_column)

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,desc,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies,fico_mean
NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
